# Module 8 – Apache Spark with Titanic Dataset

This notebook demonstrates how to use Apache Spark with the Titanic dataset using PySpark. We’ll:
- Set up Spark
- Load a dataset
- Convert it to Spark DataFrame
- Perform basic Spark operations
- Run SQL queries


## Step 1: PySpark Setup
To run this notebook, ensure `pyspark` is installed.
- If running locally, install using `pip install pyspark`
- In Google Colab, you can use the following:
```python
!pip install pyspark
```


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TitanicSparkDemo").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/29 09:44:55 WARN Utils: Your hostname, Williams-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.6.101 instead (on interface en0)
25/07/29 09:44:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/29 09:44:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Step 2: Load Titanic Dataset
We’ll use Seaborn’s version of Titanic dataset for simplicity.

In [2]:
import seaborn as sns
import pandas as pd

# Load and filter columns
df = sns.load_dataset("titanic")
df = df[['survived', 'pclass', 'age', 'sex', 'fare']].dropna()
df.head(20)

,survived,pclass,age,sex,fare
0,0,3,22.0,male,7.2500
1,1,1,38.0,female,71.2833
2,1,3,26.0,female,7.9250
3,1,1,35.0,female,53.1000
4,0,3,35.0,male,8.0500
6,0,1,54.0,male,51.8625
7,0,3,2.0,male,21.0750
8,1,3,27.0,female,11.1333
9,1,2,14.0,female,30.0708
10,1,3,4.0,female,16.7000


## Step 3: Convert to Spark DataFrame

In [6]:

df_spark = spark.createDataFrame(df)
df_spark.printSchema()
df_spark.show(10)

root
 |-- survived: long (nullable = true)
 |-- pclass: long (nullable = true)
 |-- age: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- fare: double (nullable = true)

+--------+------+----+------+-------+
|survived|pclass| age|   sex|   fare|
+--------+------+----+------+-------+
|       0|     3|22.0|  male|   7.25|
|       1|     1|38.0|female|71.2833|
|       1|     3|26.0|female|  7.925|
|       1|     1|35.0|female|   53.1|
|       0|     3|35.0|  male|   8.05|
|       0|     1|54.0|  male|51.8625|
|       0|     3| 2.0|  male| 21.075|
|       1|     3|27.0|female|11.1333|
|       1|     2|14.0|female|30.0708|
|       1|     3| 4.0|female|   16.7|
+--------+------+----+------+-------+
only showing top 10 rows


## Step 4: Spark DataFrame Operations
Basic exploration and transformations.

In [7]:
# Average age of passengers
df_spark.groupBy("pclass").avg("age").show()

# Count survivors by class
df_spark.groupBy("pclass", "survived").count().show()

+------+------------------+
|pclass|          avg(age)|
+------+------------------+
|     1|38.233440860215055|
|     3| 25.14061971830986|
|     2| 29.87763005780347|
+------+------------------+

+------+--------+-----+
|pclass|survived|count|
+------+--------+-----+
|     3|       0|  270|
|     1|       0|   64|
|     1|       1|  122|
|     2|       0|   90|
|     2|       1|   83|
|     3|       1|   85|
+------+--------+-----+



### Interpreting Spark SQL Queries on Titanic Data

In this step, we run **SQL-like queries** on the Spark DataFrame to explore the Titanic dataset:

1. **Average Age by Passenger Class**:
   - Gives insight into how passenger demographics differed across travel classes.
   - Higher-class passengers often had a higher average age.

2. **Survival Count by Class**:
   - Shows how survival was distributed between classes.
   - Useful for understanding social/economic influence on survival probability.

3. **Average Fare by Class and Gender**:
   - Highlights how ticket cost varied across classes and between male/female passengers.

These queries demonstrate how **Spark SQL** allows scalable and intuitive exploration of big data.



## Step 5: Spark SQL
We can use SQL queries by registering a temporary view.

In [13]:
df_spark.createOrReplaceTempView("titanic")


# Query 1: Average age by passenger class
avg_age_query = spark.sql("""
    SELECT Pclass, ROUND(AVG(Age), 2) AS Average_Age
    FROM titanic
    GROUP BY Pclass
    ORDER BY Pclass  
""")
print("Average Age by Passenger Class:")
avg_age_query.show()


# Query 2: Count of survivors by passenger class
survival_count_query = spark.sql("""
    SELECT Pclass, Survived, COUNT(*) AS Count
    FROM titanic
    GROUP BY Pclass, Survived
    ORDER BY Pclass, Survived
""")
print("Survival Count by Class:")
survival_count_query.show()

# # Query 3: Average Fare by class and gender
avg_fare_query = spark.sql("""
    SELECT Pclass, Sex, ROUND(AVG(Fare), 2) AS Average_Fare
    FROM titanic
    GROUP BY Pclass, Sex
    ORDER BY Pclass, Sex
""")
print("Average Fare by Class and Gender:")
avg_fare_query.show(truncate=False)

 

Average Age by Passenger Class:
+------+-----------+
|Pclass|Average_Age|
+------+-----------+
|     1|      38.23|
|     2|      29.88|
|     3|      25.14|
+------+-----------+

Survival Count by Class:
+------+--------+-----+
|Pclass|Survived|Count|
+------+--------+-----+
|     1|       0|   64|
|     1|       1|  122|
|     2|       0|   90|
|     2|       1|   83|
|     3|       0|  270|
|     3|       1|   85|
+------+--------+-----+

Average Fare by Class and Gender:
+------+------+------------+
|Pclass|Sex   |Average_Fare|
+------+------+------------+
|1     |female|107.95      |
|1     |male  |71.14       |
|2     |female|21.95       |
|2     |male  |21.11       |
|3     |female|15.88       |
|3     |male  |12.16       |
+------+------+------------+

